In [37]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
import optuna

In [28]:
data = pd.read_csv('combined_dataset.csv')

In [34]:
data

,USD to JPY,USD to EUR,USD to GBP,Close_SP,Adj Close_SP,Volume_SP,NASDAQCOM,Close_DJ,Balance_Curr_Acc,Public_debt,...,USD to EUR_lag1,USD to EUR_lag2,USD to EUR_lag3,USD to EUR_rolling_mean,USD to EUR_rolling_std,USD to GBP_lag1,USD to GBP_lag2,USD to GBP_lag3,USD to GBP_rolling_mean,USD to GBP_rolling_std
DATE,,,,,,,,,,,,,,,,,,,,,
1999-01-04,0.0089,1.1812,1.6581,1228.099976,1228.099976,8.770000e+08,2208.05,9184.27,-262.217000,5.638780e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-05,0.0090,1.1760,1.6566,1244.780029,1244.780029,7.750000e+08,2251.27,9311.19,-262.217000,5.638780e+06,...,1.1812,NaN,NaN,NaN,NaN,1.6581,NaN,NaN,NaN,NaN
1999-01-06,0.0089,1.1636,1.6547,1272.339966,1272.339966,9.869000e+08,2320.86,9544.97,-262.217000,5.638780e+06,...,1.1760,1.1812,NaN,NaN,NaN,1.6566,1.6581,NaN,NaN,NaN
1999-01-07,0.0090,1.1672,1.6495,1269.729980,1269.729980,8.630000e+08,2326.09,9537.76,-262.217000,5.638780e+06,...,1.1636,1.1760,1.1812,NaN,NaN,1.6547,1.6566,1.6581,NaN,NaN
1999-01-08,0.0090,1.1554,1.6405,1275.089966,1275.089966,9.378000e+08,2344.41,9643.32,-262.217000,5.638780e+06,...,1.1672,1.1636,1.1760,NaN,NaN,1.6495,1.6547,1.6566,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-04-19,0.0089,1.1246,1.3003,5234.180176,5234.180176,3.374700e+09,16428.82,39475.90,-468.503344,2.217556e+07,...,1.2320,1.0625,1.1164,1.185386,0.116152,1.4027,1.2531,1.4131,1.437043,0.142985
2020-04-10,0.0092,1.0951,1.2485,5234.180176,5234.180176,3.374700e+09,16428.82,39475.90,-526.065875,2.652844e+07,...,1.1246,1.2320,1.0625,1.158743,0.111709,1.3003,1.4027,1.2531,1.398357,0.153284
2021-04-02,0.0090,1.1763,1.3825,5234.180176,5234.180176,3.374700e+09,16428.82,39475.90,-822.744578,2.852787e+07,...,1.0951,1.1246,1.2320,1.129414,0.057001,1.2485,1.3003,1.4027,1.355971,0.091084


In [30]:
# Convert 'DATE' to datetime and set it as index
data['DATE'] = pd.to_datetime(data['DATE'])
data.set_index('DATE', inplace=True)


In [33]:
# Feature engineering: Lagged and rolling features
lags = 3
for target in ['USD to JPY', 'USD to EUR', 'USD to GBP']:
    for lag in range(1, lags + 1):
        data[f'{target}_lag{lag}'] = data[target].shift(lag)
    window = 7
    data[f'{target}_rolling_mean'] = data[target].rolling(window=window).mean()
    data[f'{target}_rolling_std'] = data[target].rolling(window=window).std()

In [48]:
X = data.drop(['USD to JPY', 'USD to EUR', 'USD to GBP'], axis=1)
y = data['USD to GBP']

In [49]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import optuna
tscv = TimeSeriesSplit(n_splits=5)
def objective(trial):
    param = {
        'tree_method': 'hist',  # CPU-based method
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.6, 0.7, 0.8, 0.9, 1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.01, 0.02, 0.05, 0.1]),
        'n_estimators': 100,
        'max_depth': trial.suggest_categorical('max_depth', [3, 5, 7, 9, 11]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'random_state': 42
    }
    
    rmse_list = []
    mae_list = []
    mse_list = []
    r2_list = []
    
    for train_idx, test_idx in tscv.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        model = xgb.XGBRegressor(**param)
        model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=50, verbose=False)
        preds = model.predict(X_test)
        rmse = mean_squared_error(y_test, preds, squared=False)
        mae = mean_absolute_error(y_test, preds)
        mse = mean_squared_error(y_test, preds)
        r2 = r2_score(y_test, preds)
        
        rmse_list.append(rmse)
        mae_list.append(mae)
        mse_list.append(mse)
        r2_list.append(r2)
    
    average_rmse = sum(rmse_list) / len(rmse_list)
    average_mae = sum(mae_list) / len(mae_list)
    average_mse = sum(mse_list) / len(mse_list)
    average_r2 = sum(r2_list) / len(r2_list)
    
    return {
        'average_rmse': average_rmse,
        'average_mae': average_mae,
        'average_mse': average_mse,
        'average_r2': average_r2
    }

study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: objective(trial)['average_rmse'], n_trials=300)

print("Number of finished trials:", len(study.trials))
print("Best trial:")
trial = study.best_trial
print(f"  Value (RMSE): {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# Optionally, retrieve other metrics from the best trial:
metrics = objective(study.best_trial)  # Re-evaluate the best model to get other metrics
print(f"Average MAE: {metrics['average_mae']}")
print(f"Average MSE: {metrics['average_mse']}")
print(f"Average R²: {metrics['average_r2']}")


[I 2024-05-11 23:32:14,496] A new study created in memory with name: no-name-b716c957-de9d-470c-966c-15653ca5761b
C:\Users\aghab\AppData\Local\Temp\ipykernel_24908\3639973178.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
C:\Users\aghab\AppData\Local\Temp\ipykernel_24908\3639973178.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
c:\Users\aghab\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopp

Number of finished trials: 300
Best trial:
  Value (RMSE): 0.07092850771885181
  Params: 
    lambda: 0.012203337306127568
    alpha: 0.001166529318304454
    colsample_bytree: 0.9
    subsample: 1.0
    learning_rate: 0.05
    max_depth: 11
    min_child_weight: 10


c:\Users\aghab\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
c:\Users\aghab\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
c:\Users\aghab\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
c:\Users\aghab\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stoppin

Average MAE: 0.057382284451661415
Average MSE: 0.009305240643579526
Average R²: -0.11118093441610226


### USD TO JPY
Number of finished trials: 300
Best trial:
  Value (RMSE): 0.00048282907693608136
  Params: 
    lambda: 0.35151817475639985
    alpha: 0.0010080495392771154
    colsample_bytree: 0.7
    subsample: 0.8
    learning_rate: 0.1
    max_depth: 9
    min_child_weight: 5
Average MAE: 0.00035875278124496385
Average MSE: 5.701394517688625e-07
Average R²: 0.6140709022031865

### USD to EUR
Number of finished trials: 300
Best trial:
  Value (RMSE): 0.047229947146669324
  Params: 
    lambda: 0.001018129565265905
    alpha: 0.0014249386233135613
    colsample_bytree: 0.6
    subsample: 1.0
    learning_rate: 0.1
    max_depth: 3
    min_child_weight: 10
Average MAE: 0.035937694459576036
Average MSE: 0.004237141645120639
Average R²: 0.15534653585055605

## USD to GBP
Number of finished trials: 300
Best trial:
  Value (RMSE): 0.07092850771885181
  Params: 
    lambda: 0.012203337306127568
    alpha: 0.001166529318304454
    colsample_bytree: 0.9
    subsample: 1.0
    learning_rate: 0.05
    max_depth: 11
    min_child_weight: 10
Average MAE: 0.057382284451661415
Average MSE: 0.009305240643579526
Average R²: -0.11118093441610226

USD to JPY:
RMSE: Very low at approximately 0.00048, indicating very small average errors.
MAE: Also extremely low, consistent with the RMSE.
MSE: Essentially zero, reflecting the small errors.
R²: At 0.614, this value indicates a moderate fit. While not perfect, the model explains a significant proportion of the variance in the data.
Observations:
The results suggest that the model is performing well with a decent R² score for financial time series data, which often contains a lot of noise.

USD to EUR:
RMSE: Relatively higher at 0.047.
MAE: Also higher than JPY results.
MSE: Significantly higher than JPY results.
R²: Low at 0.155, indicating the model is not explaining much of the variance in the dataset.
Observations:
These results suggest that the model struggles with EUR predictions. The relatively low R² indicates a poor fit, implying the model settings might not be capturing the underlying patterns effectively.

USD to GBP:
RMSE: Higher than EUR and JPY at 0.0709.
MAE: Consistent with the RMSE in indicating larger errors.
MSE: Reflects a higher error variance.
R²: Negative at -0.111, indicating that the model performs worse than a naive baseline model.
Observations:
The negative R² is particularly concerning as it implies the model fits the data worse than a simple average. This might be due to overfitting, inappropriate model parameters, or the model's inability to capture the complexities of the GBP data.

Recommendations for Improvement:
Parameter Adjustment: Each model might benefit from further tuning of hyperparameters, particularly exploring different settings for max_depth, learning_rate, and regularization terms. For GBP, reducing max_depth might help reduce overfitting.

Feature Engineering: More sophisticated features, possibly derived from external data sources or more advanced time-series features (like Fourier transforms or more extended lag periods), might help improve model performance, especially for EUR and GBP.

Model Complexity: Consider testing simpler or more complex models for the GBP data. Sometimes, a simpler model might generalize better.

Cross-Validation Setup: Review the time series cross-validation setup to ensure it's adequately capturing the temporal dynamics in the data, and perhaps increase the number of splits or adjust the training/testing ratio.